In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from dataclasses import dataclass
from typing import Optional, List
from fuzzywuzzy import fuzz
import evaluate
import csv
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import top_k_accuracy_score, ndcg_score

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/data/whole_dataset/mobilerec/splits/train.jsonl"
df_recommender_train = pd.read_json(input_file, lines=True)

In [4]:
df_recommender_train

,user_id,user_previous_interactions,recommended_app,negative_recommended_app,turns
0,QpiHlryI4pvcIEjc,"[{'app_name': 'Threema', 'app_package': 'ch.th...",{'app_name': 'Chug Patrol Kid Train: Ready to ...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,AlktS4aqUweiO6c2,"[{'app_name': 'Sector Strike', 'app_package': ...","{'app_name': 'Cardboard Design Lab', 'package_...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,hNdOcoE3IfbPpQQv,[],"{'app_name': 'Nasty Goats', 'package_name': 'c...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,9OJ3TBx83hlcGRW3,[{'app_name': 'RE/MAX Real Estate Search App (...,"{'app_name': 'CamScanner (License)', 'package_...","[{'app_name': 'CamScanner - PDF Scanner App', ...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,pgwkSwLUE5VPU9vQ,[],"{'app_name': 'LEGO ® Batman: Beyond Gotham', '...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
8630,FVXHRzhqHTg3AfU8,"[{'app_name': 'BodyFast Intermittent Fasting',...","{'app_name': 'FaceApp: Face Editor', 'package_...",[{'app_name': 'Photoshop Express Photo Editor'...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8631,15CoCtoYc3KXIdvy,"[{'app_name': 'Hidden Through Time', 'app_pack...","{'app_name': 'The Walking Dead: Survivors', 'p...","[{'app_name': 'Plants vs. Zombies™ Heroes', 'p...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8632,OroJNW8u3I4LORPC,"[{'app_name': 'Weedmaps: Marijuana, Cannabis, ...","{'app_name': 'Hay Day', 'package_name': 'com.s...","[{'app_name': 'Farming Simulator 18', 'package...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8633,h5nCG9WD0a6HYZYj,"[{'app_name': 'Wordament® by Microsoft', 'app_...","{'app_name': 'Socratic by Google', 'package_na...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [6]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/data/whole_dataset/mobilerec/splits/val.jsonl"
df_recommender_validation = pd.read_json(input_file, lines=True)

In [7]:
df_recommender_validation

,user_id,user_previous_interactions,recommended_app,negative_recommended_app,turns
0,7oYEjVyJoTb1Spl3,"[{'app_name': 'DEAD TRIGGER 2: Zombie Games', ...",{'app_name': 'AZ Screen Recorder - Video Recor...,"[{'app_name': 'Mobizen Screen Recorder', 'pack...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,qREr480MftU5cHmf,"[{'app_name': 'Walli - 4K Wallpapers', 'app_pa...","{'app_name': 'dfndr security: antivirus', 'pac...","[{'app_name': 'Google Authenticator', 'package...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,932Lp6L51OFUf7G4,"[{'app_name': 'Dynasty Origins: Pioneer', 'app...","{'app_name': 'Onmyoji Arena', 'package_name': ...","[{'app_name': 'Legend of Ace', 'package_name':...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,lloZoTmrDQU1UT6h,"[{'app_name': 'Star Warfare2:Payback', 'app_pa...","{'app_name': 'Good Pizza, Great Pizza', 'packa...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,3Ry4S118042YaHsQ,"[{'app_name': 'Road to Valor: World War II', '...","{'app_name': 'Pepi Wonder World: Magic Isle!',...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
1842,f9btKO9JA3uTiNNT,"[{'app_name': 'Public.com - Stocks & Crypto', ...","{'app_name': 'Mobile Legends: Bang Bang', 'pac...","[{'app_name': 'Onmyoji Arena', 'package_name':...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1843,hTIn3sXfLCYgqFl2,"[{'app_name': 'Crew Messaging and Scheduling',...","{'app_name': 'Tiles Hop: EDM Rush!', 'package_...",[{'app_name': 'Piano Music Go-EDM Piano Games'...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1844,frAqBnUgTJEoFizS,"[{'app_name': 'Google Calendar', 'app_package'...","{'app_name': 'Likee Lite - Let You Shine', 'pa...","[{'app_name': 'TikTok', 'package_name': 'com.z...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1845,uhegyP6nZH3ZDh6D,"[{'app_name': 'Broken Dawn II HD', 'app_packag...","{'app_name': 'Blockman Go', 'package_name': 'c...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [8]:
model_checkpoint = "gpt2"
bos = '<|startoftext|>'
eos = '<|endoftext|>'
pad = '<|pad|>'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint, bos_token=bos, eos_token=eos, pad_token=pad, additional_special_tokens=["computer:", "human:"])

model = GPT2LMHeadModel.from_pretrained(model_checkpoint).to(device)
model.resize_token_embeddings(len(tokenizer))
model_max_length=512

/u/spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
@dataclass
class RecommenderItem:
    prompt: str
    generation: Optional[str] = None
    
class recommenderDataset(Dataset):
    def __init__(self, data: List[RecommenderItem]):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx) -> RecommenderItem:
        return self.data[idx]

In [10]:
items_train = []
not_founds = 0

for _, row in tqdm(df_recommender_train.iterrows(), total=len(df_recommender_train)):
    prompt = bos
    found = False
    recommended = row["recommended_app"]["app_name"]
    
    for index, turn in enumerate(row["turns"]):
        computer = turn["COMPUTER"]
        
        if fuzz.partial_ratio(recommended, computer) >= 90:
            prompt += "computer: I would recommend the "
            items_train.append(RecommenderItem(prompt, recommended + " app." + eos))
            found = True
            break
        else:
            prompt += computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f" number of items: {len(items_train)}")

100%|██████████| 8635/8635 [00:29<00:00, 291.01it/s]

Could not find 19
 number of items: 8616


In [11]:
items_validation = []
not_founds = 0

for _, row in tqdm(df_recommender_validation.iterrows(), total=len(df_recommender_validation)):
    prompt = bos
    found = False
    recommended = row["recommended_app"]["app_name"]
    
    for index, turn in enumerate(row["turns"]):
        computer = turn["COMPUTER"]
        
        if fuzz.partial_ratio(recommended, computer) >= 90:
            prompt += "computer: I would recommend the "
            items_validation.append(RecommenderItem(prompt, recommended + " app." + eos))
            found = True
            break
        else:
            prompt += computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1
        
print(f"Could not find {not_founds}")
print(f" number of items: {len(items_validation)}")

  0%|          | 0/1847 [00:00<?, ?it/s]

100%|██████████| 1847/1847 [00:06<00:00, 300.58it/s]

Could not find 11
 number of items: 1836


In [12]:
items_train[70].generation

'Gear.Club - True Racing app.<|endoftext|>'

In [13]:
items_train[0].prompt

"<|startoftext|>Hi! How can I help you today?\nhuman: I am interested in children's train games that are compatible with Android.\nAlright! Any specific features you are interested in?\nhuman: I am looking for it to be interactive.\nDo you want the game to have power-ups?\nhuman: Yes, I am looking for power-ups.\nAre you interested in a free app, or would you consider paying for additional features?\nhuman: I prefer free apps.\nDo you want the game to be challenging?\nhuman: I need the game to be challenging.\nAre app bugs an issue for you?\nhuman: I prefer the app not to have bugs and glitches.\ncomputer: I would recommend the "

In [14]:
def training_collator(batch: list[recommenderDataset]): # for training a language model
    input_ids = []
    attention_masks = []
    labels = []
    for item in batch:
        prompt_tokens = tokenizer.encode(item.prompt, return_tensors="pt")[0] 
        generation_tokens = tokenizer.encode(item.generation, return_tensors="pt")[0]
        prompt_len = len(prompt_tokens)
        generation_len = len(generation_tokens)
        unused_len = model_max_length - prompt_len - generation_len
        # handling case when input is greater than tokenizer length.
        if unused_len < 0:
            prompt_start_tokens = prompt_tokens[:1]
            trimmed_prompt = prompt_tokens[unused_len * -1 + 1 :] # TODO: you could delete the prompt to reach the first |beginuser| token
            prompt_tokens = torch.cat(
                [prompt_start_tokens, trimmed_prompt], axis=0
            )
            prompt_len = len(prompt_tokens)
            unused_len = 0
        pad = torch.full([unused_len], tokenizer.pad_token_id)
        input_tokens = torch.cat(
            [prompt_tokens, generation_tokens, pad]
        )
        label = torch.cat(
            [
                torch.full(
                    [prompt_len],
                    -100,
                ),
                generation_tokens,
                torch.full([unused_len], -100),
            ]
        )
        attention_mask = torch.cat(
            [
                torch.full([prompt_len + generation_len], 1),
                torch.full([unused_len], 0),
            ]
        )
        input_ids.append(input_tokens)
        attention_masks.append(attention_mask)
        labels.append(label)

    out = {
        "input_ids": torch.stack(input_ids),
        "attention_mask": torch.stack(attention_masks),
        "labels": torch.stack(labels),
    }

    return out

In [15]:
training_args = TrainingArguments(
    output_dir="/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=0.3,#self.cfg.save_steps,
    eval_steps=0.3, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)


trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=recommenderDataset(items_train),
        eval_dataset=recommenderDataset(items_validation), #dm.datasets[DataNames.dev_language_model.value],
        data_collator=training_collator,
    )

In [16]:
trainer.train()
trainer.save_model()

Step,Training Loss,Validation Loss
1077,0.970200,0.951732
2154,0.399100,0.745969
3231,0.264000,0.729754


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [17]:
torch.cuda.empty_cache()

# Load the model and test it on the test dataset

In [4]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/data/whole_dataset/mobilerec/splits/test.jsonl"
df_recommender_test = pd.read_json(input_file, lines=True)

In [5]:
apps_training_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/data/master_app_data_V1_true.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(row["app_name"].lower())

In [ ]:
def fix_recommended_apps_names(row):
    if row["recommended_app"]["app_name"] not in all_apps:
        for app in all_apps:
            if fuzz.ratio(row["recommended_app"]["app_name"].lower(), app.lower()) > 80:
                # return app
                print(f"Found: {row['recommended_app']['app_name']} -> {app}")
        return "uno!™"
    else:
        return row["recommended_app"]["app_name"]

df_recommender_test['recommended_app_name'] = df_recommender_test.apply(fix_recommended_apps_names, axis=1)

In [6]:
def candidate_creator(row):
    np.random.seed(row.name)
    selected_values = np.random.choice(np.setdiff1d(all_apps, [row["recommended_app"]["app_name"]]), 24, replace=False)
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, row["recommended_app"]["app_name"]) 

df_recommender_test['candidate_apps'] = df_recommender_test.apply(lambda row: candidate_creator(row), axis=1)

In [7]:
df_recommender_test.iloc[0]["recommended_app"]

{'app_name': 'YouTube',
 'package_name': 'com.google.android.youtube',
 'date': '2021-12-26T00:00:00'}

In [8]:
df_recommender_test.iloc[0]["turns"]

[{'turn': 1,
  'is_rec': False,
  'user_accept_recommendation': False,
  'COMPUTER': 'Hi! How can I help you today?',
  'HUMAN': "I'm looking for a video streaming app for my Android phone. Any suggestions?"},
 {'turn': 2,
  'is_rec': False,
  'user_accept_recommendation': False,
  'COMPUTER': "Could you share some more details about what you're looking for?",
  'HUMAN': 'It’d be great if it had convenience.'},
 {'turn': 3,
  'is_rec': False,
  'user_accept_recommendation': False,
  'COMPUTER': 'Do you have any specific content rating in mind for the app?',
  'HUMAN': "I'm looking for apps suitable for teens."},
 {'turn': 4,
  'is_rec': False,
  'user_accept_recommendation': False,
  'COMPUTER': 'Are you comfortable with the app having bugs and glitches?',
  'HUMAN': 'I don’t have any problems with bugs and glitches.'},
 {'turn': 5,
  'is_rec': False,
  'user_accept_recommendation': False,
  'COMPUTER': 'Got it! What about ads?',
  'HUMAN': "I'm open to exploring apps that contain ads.

In [9]:
df_recommender_test.iloc[0]["candidate_apps"]

array(['move people', "baby panda's juice shop", 'YouTube',
       'sonic dash 2: sonic boom', 'ca notify',
       'vita - video editor & maker', 'whatsapp business',
       'jurassic world™: the game', 'gravity rider: space bike race',
       'tower defense king', 'music player - mp3 player',
       '30 day fitness challenge', 'binance: buy btc & 600+ crypto',
       'hair challenge', 'idle heroes', 'snipers vs thieves',
       'race: rocket arena car extreme', 'azur lane',
       'animal crossing: pocket camp', 'candy camera - photo editor',
       'mytherapy pill reminder', 'uber - driver: drive & deliver',
       "motogp racing '21", 'what3words', 'puzzle combat: match-3 rpg'],
      dtype='<U50')

In [10]:
@dataclass
class RecommenderItem:
    prompt: str
    generation: Optional[str] = None
    
class recommenderDataset(Dataset):
    def __init__(self, data: List[RecommenderItem]):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx) -> RecommenderItem:
        return self.data[idx]

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bos = '<|startoftext|>'
eos = '<|endoftext|>'
pad = '<|pad|>'

tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token=bos, eos_token=eos, pad_token=pad, additional_special_tokens=["computer:", "human:"], padding_side='left')

model = GPT2LMHeadModel.from_pretrained("/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/GPT2_recommender/checkpoint-3231").to(device)
model.resize_token_embeddings(len(tokenizer))
model.eval()

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50261, bias=False)
)

In [12]:
tokenizer.truncation_side

'right'

In [44]:
prompts = []
generations = []
candidate_apps = []
true_candidate_indexes = []
not_founds = 0
for _, row in df_recommender_test.iterrows():
    # creating candidate apps
    candidates = []
    for index, candidate_app in enumerate(row["candidate_apps"].tolist()):
        candidates.append(candidate_app + " app." + eos)
        if candidate_app == row["recommended_app"]["app_name"]:
            true_candidate_index = index
    prompt = bos
    found = False
    recommended = row["recommended_app"]["app_name"]
    
    for index, turn in enumerate(row["turns"]):
        computer = turn["COMPUTER"]
        
        if fuzz.partial_ratio(recommended, computer) >= 90:
            prompt += "computer: I would recommend the "
            prompts.append(prompt)
            generations.append(recommended + " app.")
            candidate_apps.append(candidates)
            true_candidate_indexes.append(true_candidate_index)
            found = True
            break
        else:
            prompt += computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"Number of prompt: {len(prompts)}")
print(f"Number of generations: {len(generations)}")
print(f"Number of candidate apps: {len(candidate_apps)}")

Could not find 3
Number of prompt: 1844
Number of generations: 1844
Number of candidate apps: 1844


In [45]:
prompts[100]

'<|startoftext|>Hello! How may I assist you today?\nhuman: I\'m interested in finding a Virtual Pet Game that works with Android. Any recommendations?\nGot it! Are there any particular features you\'re excited about?\nhuman: I’d prefer if it comes with fun elements.\nAre you looking for the game and its characters to have power-ups?\nhuman: Yes, I need the game to feature power-ups.\nDo you need the app to be user-friendly?\nhuman: Yes, I want it to be user-friendly.\nDo you have any specific content rating in mind for the app?\nhuman: I generally prefer apps with a rating of Everyone.\nBased on your preferences, I recommend trying out "Penguin Isle."\nhuman: I\'ve heard about this app you\'re recommending, but I\'m worried about potential issues. Can you tell me if users have reported any problems with battery drain, intrusive ads, or other negative points?\nBased on multiple user reviews, significant battery drain and frequent intrusive ads have been common complaints about this app.

In [46]:
generations[100]

'My Talking Hank app.'

In [47]:
candidate_apps[100]

['radio garden app.<|endoftext|>',
 "alto's odyssey app.<|endoftext|>",
 'cookie swirl world app.<|endoftext|>',
 'adobe acrobat reader: edit pdf app.<|endoftext|>',
 "alto's adventure app.<|endoftext|>",
 'My Talking Hank app.<|endoftext|>',
 "june's journey: hidden objects app.<|endoftext|>",
 'cymera - photo editor collage app.<|endoftext|>',
 'real drum: electronic drums app.<|endoftext|>',
 'walk band - multitracks music app.<|endoftext|>',
 'angry neighbor app.<|endoftext|>',
 'flex run 3d app.<|endoftext|>',
 'smashing rush : parkour action run game app.<|endoftext|>',
 'toy blast app.<|endoftext|>',
 'world of tanks blitz app.<|endoftext|>',
 'soccer live scores - sofascore app.<|endoftext|>',
 'mico: make friends, live chat and go live stream app.<|endoftext|>',
 'ramboat 2- offline action game app.<|endoftext|>',
 'infinite painter app.<|endoftext|>',
 'doordash - food delivery app.<|endoftext|>',
 'super mario run app.<|endoftext|>',
 'toca life: vacation app.<|endoftext|>',

In [37]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(prompts, generations, model, tokenizer, batch_size=8, device=device, threshold=70):
  prompt_batches = list(chunk(prompts, batch_size))
  generation_batches = list(chunk(generations, batch_size))
  max_length=512
  generation_length = 32
  correctly_predicted = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device),
                            max_new_tokens=generation_length,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    generations_predicted = generations_predicted[:, max_length:] # we only need the generation part, not the prompt part.
    decoded_generations = [tokenizer.decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True).replace(" app.", "")  for generation in generations_predicted]
    generation_batch = [generation.replace(" app.", "").replace("<|endoftext|>", "") for generation in generation_batch]
    
    correctly_predicted.extend([1 if fuzz.ratio(predicted, ground_truth) > threshold else 0 for predicted, ground_truth in zip(decoded_generations, generation_batch)])
    
  
  return correctly_predicted

In [39]:
correctly_predicted = evaluate_recommender(prompts, generations, model, tokenizer, batch_size=2, device=device, threshold=95)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

100%|██████████| 922/922 [06:07<00:00,  2.51it/s]

success_rate:  0.45336225596529284


In [38]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]
    
def convert_to_sublists(numbers, sublist_size):
    return [numbers[i:i+sublist_size] for i in range(0, len(numbers), sublist_size)]

def recommender_rank(prompts, candidate_apps, model, tokenizer, batch_size=8, device=device):
  model.eval()
  tokenizer.padding_side='left'
  tokenizer.truncation_side='left'
  max_length = 512
  prompts_ids = tokenizer(prompts, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")
  
  tokenizer.padding_side='right'
  tokenizer.truncation_side='right'
  input_ids = []
  attention_mask = []
  for index, candidate_app_elements in enumerate(candidate_apps):
    candidate_apps_ids = tokenizer(candidate_app_elements, max_length=32, truncation=True, padding="max_length", return_tensors="pt")
    for candidate_app_index in range(len(candidate_app_elements)):
      input_ids.append(torch.cat([prompts_ids["input_ids"][index], candidate_apps_ids["input_ids"][candidate_app_index]]))
      attention_mask.append(torch.cat([prompts_ids["attention_mask"][index], candidate_apps_ids["attention_mask"][candidate_app_index]]))
      
  input_ids_batches = list(chunk(input_ids, batch_size))
  attention_mask_batches = list(chunk(attention_mask, batch_size))

  scores = []
  for input_ids_batch, attention_mask_batch in tqdm(zip(input_ids_batches, attention_mask_batches), total = len(attention_mask_batches)):

    input_ids = torch.stack(input_ids_batch).to(device)
    attention_mask = torch.stack(attention_mask_batch).to(device)
    with torch.no_grad():
      model_output = model(input_ids=input_ids) # attention_mask=attention_mask

    logprobs = F.log_softmax(model_output["logits"], dim=-1)[:, max_length -1:-1, :]
    output_tokens = input_ids[:, max_length:]
    
    tokens_logprobs = torch.gather(logprobs, 2, output_tokens[:, :, None]).squeeze(-1).to(torch.float32)
    
    mask = torch.ones(tokens_logprobs.shape, dtype=torch.bool, device=device)
    for i, _output in enumerate(output_tokens):
      for j, _token in enumerate(_output):
        if _token == tokenizer.pad_token_id:
          mask[i, j] = False
          
    score = (tokens_logprobs * mask).sum(-1) / mask.sum(-1)
    scores.extend(score.to('cpu').tolist())
  
  scores = convert_to_sublists(scores, len(candidate_apps[0]))
  
  return scores

In [41]:
scores = recommender_rank(prompts, candidate_apps, model, tokenizer, batch_size=4, device=device)

100%|██████████| 11525/11525 [10:05<00:00, 19.03it/s]


In [ ]:
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

[np.float64(0.8497830802603037),
 np.float64(0.9056399132321041),
 np.float64(0.9365509761388287),
 np.float64(0.9614967462039046),
 np.float64(0.9755965292841648),
 np.float64(0.9831887201735358),
 np.float64(0.9864425162689805),
 np.float64(0.9891540130151844),
 np.float64(0.9907809110629068),
 np.float64(0.9940347071583514)]

In [52]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_apps[0]))] for index in true_candidate_indexes]

In [53]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]

[np.float64(0.8497830802603037),
 np.float64(0.8850248181224839),
 np.float64(0.9004803495758461),
 np.float64(0.9112239079659632),
 np.float64(0.9166784486319601),
 np.float64(0.91938284159265),
 np.float64(0.9204674402911316),
 np.float64(0.9213228222780776),
 np.float64(0.9218125673903294),
 np.float64(0.9227531253935328)]